In [ ]:
# Cài đặt các thư viện cần thiết
!pip install transformers torch scikit-learn pandas

# Nhập các thư viện
import pandas as pd
import torch
from sklearn.model_selection import train_test_split
from transformers import AutoTokenizer, AutoModelForSequenceClassification, Trainer, TrainingArguments
from torch.utils.data import Dataset

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 363.4/363.4 MB 3.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.8/13.8 MB 40.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 24.6/24.6 MB 28.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 883.7/883.7 kB 24.6 MB/s eta 0:00:00
   ━╸━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 25.0/664.8 MB 151.0 MB/s eta 0:00:05

In [ ]:
# Tải dữ liệu từ file CSV
data = pd.read_csv('/content/VI_IMDB.csv')

# Kiểm tra dữ liệu
print(data.head())

# Ánh xạ nhãn cảm xúc thành số (positive: 1, negative: 0)
label_map = {"positive": 1, "negative": 0}
data['label'] = data['sentiment'].map(label_map)

# Chia dữ liệu thành tập huấn luyện và tập kiểm tra
train_texts, val_texts, train_labels, val_labels = train_test_split(
    data['vi_review'].tolist(),
    data['label'].tolist(),
    test_size=0.2,
    random_state=42
)

# Kiểm tra kích thước tập dữ liệu
print(f"Tập huấn luyện: {len(train_texts)} mẫu")
print(f"Tập kiểm tra: {len(val_texts)} mẫu")

In [ ]:
# Tải tokenizer của PhoBERT
tokenizer = AutoTokenizer.from_pretrained("vinai/phobert-base")

# Token hóa tập huấn luyện và kiểm tra
train_encodings = tokenizer(
    train_texts,
    truncation=True,
    padding=True,
    max_length=128
)
val_encodings = tokenizer(
    val_texts,
    truncation=True,
    padding=True,
    max_length=128
)

# Tạo class Dataset tùy chỉnh
class SentimentDataset(Dataset):
    def __init__(self, encodings, labels):
        self.encodings = encodings
        self.labels = labels

    def __getitem__(self, idx):
        item = {key: torch.tensor(val[idx]) for key, val in self.encodings.items()}
        item['labels'] = torch.tensor(self.labels[idx])
        return item

    def __len__(self):
        return len(self.labels)

# Tạo tập dữ liệu huấn luyện và kiểm tra
train_dataset = SentimentDataset(train_encodings, train_labels)
val_dataset = SentimentDataset(val_encodings, val_labels)

In [ ]:
# Tải mô hình PhoBERT
model = AutoModelForSequenceClassification.from_pretrained("vinai/phobert-base", num_labels=2)

# Thiết lập tham số huấn luyện
training_args = TrainingArguments(
    output_dir='./results',
    num_train_epochs=3,
    per_device_train_batch_size=16,
    per_device_eval_batch_size=16,
    warmup_steps=500,
    weight_decay=0.01,
    logging_dir='./logs',
    logging_steps=10,
    eval_strategy="epoch",  # Thay evaluation_strategy thành eval_strategy
    save_strategy="epoch",
    load_best_model_at_end=True,
)

# Khởi tạo Trainer
trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=train_dataset,
    eval_dataset=val_dataset,
)

# Huấn luyện mô hình
trainer.train()

In [ ]:
# Đánh giá mô hình trên tập kiểm tra
eval_results = trainer.evaluate()
print("Kết quả đánh giá:", eval_results)

# Tính toán các chỉ số bổ sung (accuracy, precision, recall, F1)
from sklearn.metrics import accuracy_score, precision_recall_fscore_support

# Dự đoán trên tập kiểm tra
predictions = trainer.predict(val_dataset)
pred_labels = predictions.predictions.argmax(-1)

# Tính các chỉ số
accuracy = accuracy_score(val_labels, pred_labels)
precision, recall, f1, _ = precision_recall_fscore_support(val_labels, pred_labels, average='binary')

print(f"Accuracy: {accuracy:.4f}")
print(f"Precision: {precision:.4f}")
print(f"Recall: {recall:.4f}")
print(f"F1-score: {f1:.4f}")

In [ ]:
# Lưu mô hình và tokenizer
trainer.save_model("./phobert-sentiment-model")
tokenizer.save_pretrained("./phobert-sentiment-model")

# Tải mô hình để dự đoán
from transformers import pipeline

# Tạo pipeline dự đoán
sentiment_pipeline = pipeline(
    "sentiment-analysis",
    model="./phobert-sentiment-model",
    tokenizer=tokenizer
)

# Thử dự đoán trên một đánh giá mới
new_review = "Bộ phim này thật sự rất tệ và thiếu sức sống!"
result = sentiment_pipeline(new_review)
print("Dự đoán:", result)

# Chuyển đổi nhãn dự đoán
label_map_reverse = {0: "negative", 1: "positive"}
predicted_label = label_map_reverse[int(result[0]['label'].split('_')[-1])]
print(f"Cảm xúc dự đoán: {predicted_label}")

In [ ]:
from sklearn.model_selection import StratifiedKFold
import numpy as np
import torch
from torch.utils.data import Subset
from transformers import Trainer, TrainingArguments

# Assuming you have `dataset` (a Hugging Face DatasetDict) and a tokenizer
X = dataset['train']['text']
y = dataset['train']['label']

kfold = StratifiedKFold(n_splits=5, shuffle=True, random_state=42)

for fold, (train_idx, val_idx) in enumerate(kfold.split(X, y)):
    print(f"Fold {fold + 1}")

    # Subset the dataset
    train_subset = dataset['train'].select(train_idx.tolist())
    val_subset = dataset['train'].select(val_idx.tolist())

    # Tokenize
    tokenized_train = train_subset.map(lambda x: tokenizer(x['text'], truncation=True, padding='max_length'), batched=True)
    tokenized_val = val_subset.map(lambda x: tokenizer(x['text'], truncation=True, padding='max_length'), batched=True)

    # Set format for PyTorch
    tokenized_train.set_format(type='torch', columns=['input_ids', 'attention_mask', 'label'])
    tokenized_val.set_format(type='torch', columns=['input_ids', 'attention_mask', 'label'])

    # Load model
    model = AutoModelForSequenceClassification.from_pretrained("vinai/phobert-base", num_labels=2)

    # Define Trainer
    training_args = TrainingArguments(
        output_dir=f"./results_fold{fold}",
        num_train_epochs=3,
        per_device_train_batch_size=16,
        per_device_eval_batch_size=64,
        evaluation_strategy="epoch",
        save_strategy="no",
        logging_dir='./logs',
        logging_steps=10,
    )

    trainer = Trainer(
        model=model,
        args=training_args,
        train_dataset=tokenized_train,
        eval_dataset=tokenized_val,
        tokenizer=tokenizer,
        compute_metrics=compute_metrics,  # Define accuracy/F1 metric separately
    )

    trainer.train()
    trainer.evaluate()
